# Bielik v3 4.5B - Function Calling

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

/home/kacper/Documents/Learn/ai_playground/models/bielik/bielik_v3_func_calling/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Sprawdzenie dostępności GPU

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

if device == "cpu":
    print("You dont't have access to gpu")

Device: cuda


## Prepare Environment

In [3]:
import os
from getpass import getpass

hf_token = getpass("Enter your Hugging Face token.")
enable_hf = bool(hf_token)
print(f"Is Hugging Face enabled? {enable_hf}")

if enable_hf:
    os.environ["HF_TOKEN"] = hf_token

Is Hugging Face enabled? True


## Download model and tokenizer

In [4]:
model_name = "speakleash/Bielik-4.5B-v3.0-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    device_map="auto"
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.31s/it]
Some parameters are on the meta device because they were offloaded to the cpu.


### First try

In [7]:
prompt = "Przepis na jajecznicę w dwóch zdaniach"
messages = [{"role": "user", "content": prompt}]
input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt").to(device)
outputs = model.generate(input_ids, max_new_tokens=1000)
response = tokenizer.batch_decode(outputs)[0]
print(response)

<s><|im_start|> user
Przepis na jajecznicę w dwóch zdaniach<|im_end|> 

1. Na patelni rozgrzej masło, dodaj posiekaną cebulę i smaż, aż zmięknie.
2. Wbij jajka, dopraw solą i pieprzem, mieszaj, aż się zetną.<|im_end|>


## Define function

In [9]:
def get_weather(city):
    """Sprawdza pogodę w podanym mieście."""
    weather = {
        "warszawa": {"temperatura": "12°C", "opis": "słonecznie"},
        "kraków": {"temperatura": "10°C", "opis": "zachmurzenie"},
        "gdańsk": {"temperatura": "18°C", "opis": "deszczowo"},
        "bydgoszcz": {"temperatura": "23°C", "opis": "słonecznie"}
    }

    city = city.lower()
    if city in weather:
        return weather[city]
    else:
        return {"błąd": f"Nie znaleziono pogody dla miasta {city}"}

In [10]:
print(get_weather("Warszawa"))

{'temperatura': '12°C', 'opis': 'słonecznie'}


## Funtion format for LLM

In [11]:
import json

tools = [
    {
        "name": "get_weather",
        "description": "Sprawdza aktualną pogodę w podanym polskim mieście",
        "parameters": {
            "type": "object",
            "properties": {
                "city": "string",
                "description": "Nazwa miasta w Polsce, np. Warszawa"
            }
        },
        "required": ["city"]
    }
]

tools_json = json.dumps(tools, ensure_ascii=False)
print("Function definition for model:")
print(tools_json)

Function definition for model:
[{"name": "get_weather", "description": "Sprawdza aktualną pogodę w podanym polskim mieście", "parameters": {"type": "object", "properties": {"city": "string", "description": "Nazwa miasta w Polsce, np. Warszawa"}}, "required": ["city"]}]


## Query to model with function definition

In [17]:
prompt = "Jaka jest pogoda w Bydgoszczy?"

system = "Jesteś pomocnym asystentem AI. Gdy potrzebujesz dodatkowych informacji, możesz używać dostępnych funkcji."

main_prompt = f"{prompt}\n\nDostępne funkcje: {tools_json}"
 
formatted_prompt = f"<|im_start|> system\n{system}<|im_end|>\n<|im_start|> user\n{main_prompt}<|im_end|>\n<|im_start|> assistant\n"

inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

outputs = model.generate(inputs["input_ids"], max_new_tokens=500)

raw_response = tokenizer.decode(outputs[0], skip_special_tokens=False)
print("Surowa odpowiedź modelu:")
print(raw_response)

Surowa odpowiedź modelu:
<s><|im_start|>  system
Jesteś pomocnym asystentem AI. Gdy potrzebujesz dodatkowych informacji, możesz używać dostępnych funkcji.<|im_end|> 
<|im_start|>  user
Jaka jest pogoda w Bydgoszczy?

Dostępne funkcje: [{"name": "get_weather", "description": "Sprawdza aktualną pogodę w podanym polskim mieście", "parameters": {"type": "object", "properties": {"city": "string", "description": "Nazwa miasta w Polsce, np. Warszawa"}}, "required": ["city"]}]<|im_end|> 
<|im_start|>  assistant
Aby uzyskać informacje o pogodzie w Bydgoszczy, użyję funkcji `get_weather` z miastem "Bydgoszcz".

<tool_call> {"name": "get_weather", "arguments": {"city": "Bydgoszcz"}}</tool_call><|im_end|>


## Extract function call from response

In [27]:
import re

def get_function_calling(text):
    pattern = r"<tool_call>\s*(.*?)\s*</tool_call>"
    match = re.search(pattern, text, re.DOTALL)
    
    if match:
        try:
            execute = json.loads(match.group(1))
            if "name" in execute:
                execute["function"] = execute["name"]
            return execute
        except:  # noqa: E722
            pass
        
    return None

func_execution = get_function_calling(raw_response)
print("\nWyodrębnione wywołanie funkcji:")
print(json.dumps(func_execution, indent=2, ensure_ascii=False) if func_execution else "Nie znaleziono wywołania funkcji")


Wyodrębnione wywołanie funkcji:
{
  "name": "get_weather",
  "arguments": {
    "city": "Bydgoszcz"
  },
  "function": "get_weather"
}


## Function execution

In [33]:
if func_execution and "function" in func_execution and "arguments" in func_execution:
    function_name = func_execution["function"]
    arguments = func_execution["arguments"]

    print(f"\nWykonuję funkcję: {function_name}")
    print(f"Z argumentami: {json.dumps(arguments, ensure_ascii=False)}")

    try:
        # Używamy eval() aby wywołać funkcję z argumentami
        result = eval(f"{function_name}(**{arguments})")
    except Exception as e:
        result = {"błąd": str(e)}
        print(f"\nBłąd: {str(e)}")
else:
    result = None
    print("\nNie można wykonać funkcji - brak poprawnego wywołania.")

if result is not None:
    print("\nWynik funkcji:")
    print(json.dumps(result, indent=2, ensure_ascii=False))


Wykonuję funkcję: get_weather
Z argumentami: {"city": "Bydgoszcz"}

Wynik funkcji:
{
  "temperatura": "23°C",
  "opis": "słonecznie"
}


## Send funtion execution result to llm

In [35]:
if result is not None:
    # Tworzymy nowy prompt zawierający całą historię
    final_prompt = (
        f"{raw_response}\n"
        f"<|im_start|> tool\n"
        f"{json.dumps(result, ensure_ascii=False)}<|im_end|>\n"
        f"<|im_start|> assistant\n"
    )

    final_inputs = tokenizer(final_prompt, return_tensors="pt").to(device)

    final_outputs = model.generate(final_inputs["input_ids"], max_new_tokens=500)

    final_response = tokenizer.decode(final_outputs[0], skip_special_tokens=False)
    print("\nOstateczna odpowiedź modelu (z wynikiem funkcji):")
    print(final_response)

    print("\n\n-------------------------------\n\n")
    # Wyodrębniamy tylko ostatnią odpowiedź asystenta
    import re
    pattern = r'<\|im_start\|\>\s+assistant\n(.*?)<\|im_end\|>'
    matches = re.findall(pattern, final_response, re.DOTALL)

    if matches:
        last_assistant = matches[-1].strip()  # Bierzemy ostatnie dopasowanie
        print("\nOstateczna odpowiedź (oczyszczona):")
        print(last_assistant)
    else:
        print("\nNie znaleziono odpowiedzi asystenta")


Ostateczna odpowiedź modelu (z wynikiem funkcji):
<s><s><|im_start|>   system
Jesteś pomocnym asystentem AI. Gdy potrzebujesz dodatkowych informacji, możesz używać dostępnych funkcji.<|im_end|>  
<|im_start|>   user
Jaka jest pogoda w Bydgoszczy?

Dostępne funkcje: [{"name": "get_weather", "description": "Sprawdza aktualną pogodę w podanym polskim mieście", "parameters": {"type": "object", "properties": {"city": "string", "description": "Nazwa miasta w Polsce, np. Warszawa"}}, "required": ["city"]}]<|im_end|>  
<|im_start|>   assistant
Aby uzyskać informacje o pogodzie w Bydgoszczy, użyję funkcji `get_weather` z miastem "Bydgoszcz".

<tool_call>  {"name": "get_weather", "arguments": {"city": "Bydgoszcz"}}</tool_call><|im_end|> 
<|im_start|>  tool
{"temperatura": "23°C", "opis": "słonecznie"}<|im_end|> 
<|im_start|>  assistant
Aktualna pogoda w Bydgoszczy jest słoneczna z temperaturą 23°C.<|im_end|>


-------------------------------



Ostateczna odpowiedź (oczyszczona):
Aktualna pogod